## Импорты и проч

In [1]:
# Импортируем необходимые библиотеки и функции
import os
import json
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf
import time

from baskerville import seqnn, gene as bgene
from borzoi_helpers import process_sequence, predict_tracks  # предполагается, что эти функции доступны

# Отключаем лишние предупреждения TensorFlow
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


2025-02-19 00:43:02.654227: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 00:43:02.726358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-19 00:43:02.726414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-19 00:43:02.726531: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 00:43:02.754601: I tensorflow/core/platform/cpu_feature_g

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)
!nvcc -V

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2.14.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
%%bash

#Download model weights (data fold 3, 4 replicates)
for rep in f3c0,f0 f3c1,f1 f3c2,f2 f3c3,f3; do IFS=","; set -- $rep; 
  mkdir -p "saved_models/$1/train"
  local_model="saved_models/$1/train/model0_best.h5"
  if [ -f "$local_model" ]; then
    echo "$1 model already exists."
  else
    wget --progress=bar:force "https://storage.googleapis.com/seqnn-share/borzoi/$2/model0_best.h5" -O "$local_model"
  fi
done

#Download and uncompress annotation files
mkdir -p hg38/genes/gencode41
mkdir -p hg38/genes/polyadb

if [ -f hg38/genes/gencode41/gencode41_basic_nort.gtf ]; then
  echo "Gene annotation already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_nort.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_nort.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_nort_protein.gtf ]; then
  echo "Gene annotation (no read-through, protein-coding) already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_nort_protein.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_nort_protein.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein.gtf ]; then
  echo "Gene annotation (protein-coding) already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_protein.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_tss2.bed ]; then
  echo "TSS annotation already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_tss2.bed.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_tss2.bed
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein_splice.csv.gz ]; then
  echo "Splice site annotation already exist."
else
  wget https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein_splice.csv.gz -O hg38/genes/gencode41/gencode41_basic_protein_splice.csv.gz
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein_splice.gff ]; then
  echo "Splice site annotation already exist."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein_splice.gff.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_protein_splice.gff
fi

if [ -f hg38/genes/polyadb/polyadb_human_v3.csv.gz ]; then
  echo "PolyA site annotation already exist."
else
  wget https://storage.googleapis.com/seqnn-share/helper/polyadb_human_v3.csv.gz -O hg38/genes/polyadb/polyadb_human_v3.csv.gz
fi

#Download and index hg38 genome
mkdir -p hg38/assembly/ucsc

if [ -f hg38/assembly/ucsc/hg38.fa ]; then
  echo "Human genome FASTA already exists."
else
  wget -O - http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz | gunzip -c > hg38/assembly/ucsc/hg38.fa
fi


f3c0 model already exists.
f3c1 model already exists.
f3c2 model already exists.
f3c3 model already exists.
Gene annotation already exists.
Gene annotation (no read-through, protein-coding) already exists.
lready exists.n (protein-coding) a
TSS annotation already exists.
Splice site annotation already exist.
tation already exist.
PolyA site annotation already exist.
Human genome FASTA already exists.


## Препроцессинг

Код для понимания какой индекс отвечает за какую клеточную линию

In [1]:
import pandas as pd

targets_file = 'targets_human.txt'

col_names = [
    "identifier",
    "file_id",
    "file_path",
    "clip",
    "clip_soft",
    "scale",
    "sum_stat",
    "strand_pair",
    "description",
]
targets_df = pd.read_csv(
    targets_file,
    sep="\t",
    index_col=0,
    names=col_names
)

display(targets_df.head(10))

# Эксперименты, которые хотим отфильтровать по file_path (3-й столбец)
experiment_ids = [
    "ENCSR892LBU",
    "ENCSR357BYU",
    "ENCSR763OMY",
    "ENCSR071DYD",
    "ENCSR045GTF",
]

# Фильтруем строки, у которых file_path содержит любую из нужных строк
def matches_exps(fp):
    return any(exp_id in fp for exp_id in experiment_ids)

filtered_df = targets_df[ targets_df['file_path'].apply(matches_exps) ].copy()

# Добавим явный local_index (до слайса)
filtered_df['local_index'] = range(len(filtered_df))

print("Найдены эксперименты (отфильтрованы строки):")
display(filtered_df[['file_path','clip','clip_soft','scale','sum_stat','strand_pair','description','local_index']])

# Список индексов (относительно targets_df) которые нужны для модели
target_index = filtered_df.index
print("\nСписок глобальных индексов таргетов:", target_index.tolist())


,file_id,file_path,clip,clip_soft,scale,sum_stat,strand_pair,description
identifier,,,,,,,,
NaN,identifier,file,clip,clip_soft,scale,sum_stat,strand_pair,description
0.0,CNhs10608+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,1,CAGE:Clontech Human Universal Reference Total ...
1.0,CNhs10608-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,0,CAGE:Clontech Human Universal Reference Total ...
2.0,CNhs10610+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,3,CAGE:SABiosciences XpressRef Human Universal T...
3.0,CNhs10610-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,2,CAGE:SABiosciences XpressRef Human Universal T...
4.0,CNhs10612+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,5,CAGE:Universal RNA - Human Normal Tissues Bioc...
5.0,CNhs10612-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,4,CAGE:Universal RNA - Human Normal Tissues Bioc...
6.0,CNhs10615+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,7,"CAGE:adipose tissue, adult, pool1"
7.0,CNhs10615-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,6,"CAGE:adipose tissue, adult, pool1"


Найдены эксперименты (отфильтрованы строки):


,file_path,clip,clip_soft,scale,sum_stat,strand_pair,description,local_index
identifier,,,,,,,,
6396.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6397,RNA:lung tissue female adult (47 years),0
6397.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6396,RNA:lung tissue female adult (47 years),1
6423.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6424,RNA:pancreas tissue female child (16 years),2
6424.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6423,RNA:pancreas tissue female child (16 years),3
6730.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6731,RNA:left lobe of liver tissue male adult (45 y...,4
6731.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6730,RNA:left lobe of liver tissue male adult (45 y...,5
7240.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,7241,RNA:adrenal gland tissue female adult (41 years),6
7241.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,7240,RNA:adrenal gland tissue female adult (41 years),7
7396.0,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,7397,RNA:kidney tissue female adult (47 years),8



Список глобальных индексов таргетов: [6396.0, 6397.0, 6423.0, 6424.0, 6730.0, 6731.0, 7240.0, 7241.0, 7396.0, 7397.0]


In [4]:
# --- Получения списка интервалов и их мерж ---

import pandas as pd
import gzip
from tqdm import tqdm

bed_file_path = '../data/sequences_human.bed.gz'
bed_data = pd.read_csv(
    bed_file_path,
    sep='\t',
    header=None,
    names=["chrom", "start", "end", "fold"],
    compression='gzip'
)

folds_to_process = ['fold3']  # Можно изменить
filtered_bed_data = bed_data[ bed_data['fold'].isin(folds_to_process) ]
grouped = filtered_bed_data.groupby("chrom")

print(len(filtered_bed_data),"строк до мерджа.")

def merge_intervals(intervals):
    intervals.sort(key=lambda x: x[0])
    merged = []
    for st, en in intervals:
        if not merged or st > merged[-1][1]:
            merged.append([st, en])
        else:
            merged[-1][1] = max(merged[-1][1], en)
    return merged

merged_intervals_list = []
unique_chroms = filtered_bed_data['chrom'].nunique()
for chrom, group in tqdm(filtered_bed_data.groupby("chrom"), desc="Мержим интервалы", total=unique_chroms):
    intervals = group[['start','end']].values.tolist()
    merged = merge_intervals(intervals)
    fold_val = group['fold'].iloc[0]
    for st, en in merged:
        merged_intervals_list.append({'chrom': chrom, 'start': st, 'end': en, 'fold': fold_val})

filtered_bed_data = pd.DataFrame(merged_intervals_list)
print("После мерджа:", len(filtered_bed_data))
display(filtered_bed_data.head())


6888 строк до мерджа.


Мержим интервалы: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1037.95it/s]

После мерджа: 46


,chrom,start,end,fold
0,chr1,143479625,143823752,fold3
1,chr1,227321006,228550247,fold3
2,chr11,25440674,48748592,fold3
3,chr11,48896195,49191149,fold3
4,chr11,54525074,54820028,fold3


## Проверка данных

Убедимся что все записи влезут во входящее окно модели

In [6]:
# Находим максимальную длину последовательности
max_sequence_length = 0

for index, row in filtered_bed_data.iterrows():
    start = row['start']
    end = row['end']
    
    # Вычисляем длину текущей последовательности
    sequence_length = end - start
    
    # Обновляем максимальную длину, если текущая больше
    if sequence_length > max_sequence_length:
        max_sequence_length = sequence_length

print(f"Максимальная длина последовательности: {max_sequence_length} нуклеотидов")


Максимальная длина последовательности: 67957002 нуклеотидов


Проверка на пересечения

In [7]:
import pandas as pd
from tqdm import tqdm

overlap_found = False

# Получаем список уникальных хромосом
chromosomes = filtered_bed_data['chrom'].unique()

# Используем tqdm для отображения прогресса
for chrom in tqdm(chromosomes, desc="Checking for overlaps"):
    # Фильтруем по текущей хромосоме
    group = filtered_bed_data[filtered_bed_data['chrom'] == chrom]
    
    # Сортируем записи по старту
    sorted_group = group.sort_values('start')
    
    # Инициализируем предыдущую запись
    prev_row = None
    
    # Проходим по отсортированным записям
    for idx, row in sorted_group.iterrows():
        if prev_row is not None:
            # Проверяем пересечение
            if row['start'] < prev_row['end']:
                print(f"Пересечение на {chrom}: {prev_row[['start', 'end']].to_dict()} и {row[['start', 'end']].to_dict()}")
                overlap_found = True
        prev_row = row

if not overlap_found:
    print("Пересечений не обнаружено в filtered_bed_data.")


Checking for overlaps: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1105.77it/s]

Пересечений не обнаружено в filtered_bed_data.


In [8]:
import pandas as pd
import gzip

# Загрузим данные из bed-файла
bed_file_path = '../data/sequences_human.bed.gz'
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, 
                           names=["chrom", "start", "end", "fold"])

# Фильтруем данные по fold3 и fold4
#folds_to_process = ['fold3', 'fold4']
folds_to_process = ['fold3']
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Выведем список уникальных хромосом
unique_chroms = filtered_bed_data['chrom'].unique()
print("Уникальные хромосомы в выборке:")
print(unique_chroms)
print(f"Количество уникальных хромосом: {len(unique_chroms)}")

# Для каждой хромосомы находим минимальный start и максимальный end
chrom_stats = filtered_bed_data.groupby("chrom").agg(min_start=('start', 'min'),
                                                      max_end=('end', 'max'))
print("\nМинимальный start и максимальный end по хромосомам:")
print(chrom_stats)

if len(unique_chroms) > 1:
    print("\nВыборка содержит более одной хромосомы.")
else:
    print("\nВыборка содержит только одну хромосому.")


Уникальные хромосомы в выборке:
['chr5' 'chr11' 'chr6' 'chr9' 'chr13' 'chr2' 'chr8' 'chr7' 'chr17' 'chrX'
 'chr18' 'chr12' 'chr15' 'chr19' 'chr1' 'chr16' 'chr20']
Количество уникальных хромосом: 17

Минимальный start и максимальный end по хромосомам:
       min_start    max_end
chrom                      
chr1   143479625  228550247
chr11   25440674   56541083
chr12   32992234   34368994
chr13   40653298  102135774
chr15   20168638   34675550
chr16   33491584   33884884
chr17   81799133   83225066
chr18   13485786   15206757
chr19    3108622    9156817
chr2    68747521  129505661
chr20   30186694   30383302
chr5     8770274  103148800
chr6    99603327  170690035
chr7       10000   74778699
chr8     8158857   55824055
chr9    80588528  138217638
chrX     3230043   58109050

Выборка содержит более одной хромосомы.


## Модель

In [6]:
# --- ЯЧЕЙКА 5: Инициализация модели ---

from baskerville import seqnn
import json


params_file = 'params_pred.json'
with open(params_file) as f:
    params = json.load(f)

params_model = params['model']
params_train = params['train']

model_file = 'saved_models/f3c0/train/model0_best.h5'
rc = False  # мы не усредняем +/-, если в таблице уже есть coverage+ и coverage- каналы
n_reps = 1

# Важный момент: берем target_index из предыдущего фильтра (ячейка 2)
#   (где мы собрали нужные эксперименты)
print("Используем следующие таргет-индексы:", target_index.tolist())

seqnn_model = seqnn.SeqNN(params_model)
seqnn_model.restore(model_file, 0)
seqnn_model.build_slice(target_index)  # <-- выбираем только нужные каналы
seqnn_model.build_ensemble(rc, [0])

models = [seqnn_model]
print("Модель загружена и готова. Срез по", len(target_index), "каналам.")

# Параметры
print("stride =", seqnn_model.model_strides[0],
      "crop =", seqnn_model.target_crops[0],
      "tlen =", seqnn_model.target_lengths[0])


Используем следующие таргет-индексы: [6396.0, 6397.0, 6423.0, 6424.0, 6730.0, 6731.0, 7240.0, 7241.0, 7396.0, 7397.0]


2025-02-20 17:32:25.432439: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23390 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:01.0, compute capability: 7.0
2025-02-20 17:32:25.433137: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30965 MB memory:  -> device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:02.0, compute capability: 7.0


TypeError: Exception encountered when calling layer "tf.gather" (type TFOpLambda).

Value passed to parameter 'indices' has DataType float64 not in list of allowed values: int16, int32, int64

Call arguments received by layer "tf.gather" (type TFOpLambda):
  • params=tf.Tensor(shape=(None, 16352, 7611), dtype=float32)
  • indices=Float64Index([6396.0, 6397.0, 6423.0, 6424.0, 6730.0, 6731.0, 7240.0, 7241.0,
              7396.0, 7397.0],
             dtype='float64', name='identifier')
  • validate_indices=None
  • axis=-1
  • batch_dims=0
  • name=None

In [8]:
def debug_model(model, seq_len=524288):
    """
    Отладочная функция для проверки работы модели.
    Выводит:
      - модельную сводку (summary)
      - ключевые параметры: stride, crop, target_length
      - форму входного (dummy) массива
      - форму выходных предсказаний (для прямой и реверс-комплементарной последовательностей)
    """
    print("=== Model Summary ===")
    model.model.summary()
    
    print("\n=== Model Parameters ===")
    print(f"stride         = {model.model_strides[0]}")
    print(f"crop           = {model.target_crops[0]}")
    print(f"target_length  = {model.target_lengths[0]}")
    
    # Создадим dummy one-hot последовательность
    # Каждая строка будет ровно одной единицей на 4 элементах.
    dummy_seq = np.zeros((seq_len, 4), dtype=np.float32)
    for i in range(seq_len):
        dummy_seq[i, np.random.randint(0, 4)] = 1.0
    print("\nDummy sequence shape:", dummy_seq.shape)
    
    # Предсказание для прямой цепи
    dummy_pred = predict_tracks([model], dummy_seq)
    print("\n=== Raw Prediction Output (прямая цепь) ===")
    print("dummy_pred.shape =", dummy_pred.shape)
    # Ожидается форма вроде: [1, L_out, X, C]. Попробуем взять replicate=0 и strand=0:
    try:
        y_plus = dummy_pred[0, :, 0, :]
        print("После индексирования y_plus.shape =", y_plus.shape)
    except Exception as e:
        print("Ошибка индексирования y_plus:", e)
    
    # Предсказание для реверс-комплементарного входа
    def reverse_complement_onehot(seq_onehot):
        rev = np.flip(seq_onehot, axis=0)
        revcomp = rev[:, [3,2,1,0]]
        return revcomp

    dummy_seq_rc = reverse_complement_onehot(dummy_seq)
    dummy_pred_rc = predict_tracks([model], dummy_seq_rc)
    print("\n=== Raw Prediction Output (реверс-компл.) ===")
    print("dummy_pred_rc.shape =", dummy_pred_rc.shape)
    try:
        y_minus = dummy_pred_rc[0, :, 0, :]
        print("После индексирования y_minus.shape =", y_minus.shape)
    except Exception as e:
        print("Ошибка индексирования y_minus:", e)
    
    # Можно вывести также максимальные и минимальные значения предсказания
    print("\n=== Примеры статистики выходного тензора ===")
    print("Прямая цепь: min =", np.min(y_plus), "max =", np.max(y_plus))
    print("Реверс цепь: min =", np.min(y_minus), "max =", np.max(y_minus))


# Запускаем отладку модели:
debug_model(seqnn_model)


=== Model Summary ===
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequence (InputLayer)       [(None, 524288, 4)]       0         
                                                                 
 model_1 (Functional)        (None, 16352, 7611)       185917723 
                                                                 
 tf.compat.v1.gather (TFOpL  (None, 16352, 89)         0         
 ambda)                                                          
                                                                 
Total params: 185917723 (709.22 MB)
Trainable params: 185892699 (709.12 MB)
Non-trainable params: 25024 (97.75 KB)
_________________________________________________________________

=== Model Parameters ===
stride         = 32
crop           = 16
target_length  = 16352

Dummy sequence shape: (524288, 4)


2025-02-19 00:44:10.199745: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902



=== Raw Prediction Output (прямая цепь) ===
dummy_pred.shape = (1, 1, 16352, 89)
После индексирования y_plus.shape = (1, 89)

=== Raw Prediction Output (реверс-компл.) ===
dummy_pred_rc.shape = (1, 1, 16352, 89)
После индексирования y_minus.shape = (1, 89)

=== Примеры статистики выходного тензора ===
Прямая цепь: min = 0.000743 max = 0.1045
Реверс цепь: min = 0.000275 max = 0.05405


In [9]:
fasta_path = "hg38/assembly/ucsc/hg38.fa"
fasta_index = pysam.Fastafile(fasta_path)

chr_sizes = {
    chrom: fasta_index.get_reference_length(chrom)
    for chrom in fasta_index.references
}

print("Пример ключей chr_sizes:", list(chr_sizes.keys())[:5])


Пример ключей chr_sizes: ['chr1', 'chr10', 'chr11', 'chr11_KI270721v1_random', 'chr12']


# Инференс

In [10]:
center_len = 196_608
context_len = 163_840

all_windows = []
for _, row in filtered_bed_data.iterrows():
    chrom = row['chrom']
    start_merged = row['start']
    end_merged   = row['end']
    curr_start = start_merged
    while curr_start < end_merged:
        curr_end = curr_start + center_len
        if curr_end > end_merged:
            curr_end = end_merged
        all_windows.append((chrom, curr_start, curr_end))
        curr_start = curr_end

print(f"Всего сформировано {len(all_windows)} окон.")


Всего сформировано 1785 окон.


In [20]:
# --- ЯЧЕЙКА 8: Предсоздание bedGraph файлов ---

output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)

folds_str = "_".join(folds_to_process)
chains = ["st+", "st-"]

for bed_name in channels_dict.keys():
    for chain in chains:
        file_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        with open(file_path, "w") as f:
            f.write(f"track type=bedGraph name=\"{bed_name} {chain}\"\n")

print("Предсозданы файлы .bedGraph:")
for bed_name in channels_dict.keys():
    for chain in chains:
        fp = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        print(fp)


Предсозданы файлы .bedGraph:
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF242BWW_lung.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF242BWW_lung.bedGraph


In [17]:
def reverse_complement_onehot(seq_onehot):
    """Обращаем (L,4) → (L,4) реверс-комплемент."""
    rev = np.flip(seq_onehot, axis=0)
    revcomp = rev[:, [3,2,1,0]]
    return revcomp


def process_window(chrom, center_start, center_end, context_len, models):
    """
    Выполняем:
      1) Берём вход с запасом context_len.
      2) Предсказываем +цепь (y_plus).
      3) Предсказываем -цепь (y_minus), через reverse complement.
      4) Вырезаем из выходных массивов только центральный диапазон,
         учитывая crop.
      5) Возвращаем (center_pred_plus, center_pred_minus),
         размер [ (center_end-center_start)//stride , num_targets ].
         Либо (None, None), если что-то пошло не так.
    """

    input_start = max(0, center_start - context_len)
    input_end   = min(chr_sizes[chrom], center_end + context_len)

    # Получаем one-hot
    seq_plus = process_sequence(fasta_index, chrom, input_start, input_end)
    # print(f"[DEBUG] chrom={chrom}, input_start={input_start}, input_end={input_end}, "
    #       f"seq_plus_shape={seq_plus.shape if seq_plus is not None else None}, "
    #       f"chr_size={chr_sizes[chrom]}")
    if seq_plus is None or seq_plus.shape[0] == 0:
    #    print("[DEBUG] seq_plus is None или имеет длину 0 → return (None, None)")
        return None, None

    # Предсказание для + цепи
    y_plus = predict_tracks(models, seq_plus)  # ожидается [1, L_out, strands, C]
    # Индексируем явно: берем replicate=0, strand=0 (если strand=2, тогда берите 0 или 1 по желанию)
    y_plus = y_plus[0, 0, :, :]  # теперь [L_out, C]

    # Предсказание для - цепи
    seq_minus = reverse_complement_onehot(seq_plus)
    y_minus = predict_tracks(models, seq_minus)
    y_minus = y_minus[0, 0, :, :]

    L_out = y_plus.shape[0]
    #print(f"[DEBUG] y_plus.shape={y_plus.shape}, y_minus.shape={y_minus.shape}, L_out={L_out}")

    # Обрезка по crop
    offset_nt = stride * crop  # например, 16 * 32 = 512
    out_start_on_input = offset_nt
    out_end_on_input   = offset_nt + stride*(L_out - 2*crop)

    # Вычислим позицию [center_start, center_end] в координатах входа
    center_start_in_input = center_start - input_start
    center_end_in_input   = center_end   - input_start

    #print(f"[DEBUG] crop={crop}, stride={stride}, offset_nt={offset_nt}")
    #print(f"[DEBUG] out_start_on_input={out_start_on_input}, out_end_on_input={out_end_on_input}, "
    #      f"center_start_in_input={center_start_in_input}, center_end_in_input={center_end_in_input}")

    # Пересечение
    slice_start = max(center_start_in_input, out_start_on_input)
    slice_end   = min(center_end_in_input, out_end_on_input)
    if slice_end <= slice_start:
        #print(f"[DEBUG] slice_end <= slice_start => return None, None: slice_start={slice_start}, slice_end={slice_end}")
        return None, None

    out_slice_start = int((slice_start - out_start_on_input)//stride)
    out_slice_end   = int(np.ceil((slice_end - out_start_on_input)/stride))

    # Проверим границы
    if out_slice_start < 0:
        out_slice_start = 0
    if out_slice_end > (L_out - 2*crop):
        out_slice_end = L_out - 2*crop
    if out_slice_end <= out_slice_start:
        #print(f"[DEBUG] out_slice_end <= out_slice_start => return None, None: "
        #      f"out_slice_start={out_slice_start}, out_slice_end={out_slice_end}")
        return None, None

    # Сдвигаем на crop (т.к. первые crop "ячейки" модели вырезаются)
    out_slice_start_full = crop + out_slice_start
    out_slice_end_full   = crop + out_slice_end

    center_pred_plus = y_plus[out_slice_start_full : out_slice_end_full, :]
    center_pred_minus = y_minus[out_slice_start_full : out_slice_end_full, :]

    #print(f"[DEBUG] out_slice_start_full={out_slice_start_full}, out_slice_end_full={out_slice_end_full}, "
    #      f"center_pred_plus.shape={center_pred_plus.shape}, center_pred_minus.shape={center_pred_minus.shape}")

    return center_pred_plus, center_pred_minus


def run_inference(all_windows, models):
    """
    Для каждого (chrom, cstart, cend) → предсказываем center_pred_plus / center_pred_minus
    и записываем bedGraph, если всё ок.
    """
    start_time = time.time()
    print(f"Начинаем инференс по {len(all_windows)} окнам.")

    for (chrom, cstart, cend) in tqdm(all_windows, desc="Inference windows"):
        center_pred_plus, center_pred_minus = process_window(chrom, cstart, cend, context_len, models)

        if center_pred_plus is None:
            print("+ косячный")
        if center_pred_minus is None:
            print("- косячный")

        if center_pred_plus is None or center_pred_minus is None:
            continue

        length_out = center_pred_plus.shape[0]
        for bed_name, channel_ix in channels_dict.items():
            # + цепь
            if channel_ix < center_pred_plus.shape[1]:
                vals_plus = center_pred_plus[:, channel_ix]
                file_path_plus = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_st+_{bed_name}.bedGraph")
                
                with open(file_path_plus, "a") as fp:
                    for i, val in enumerate(vals_plus):
                        pos_start = cstart + i*stride
                        pos_end   = pos_start + stride
                        if pos_end > cend:
                            pos_end = cend
                        if pos_end <= pos_start:
                            break
                        fp.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

                #print(f"[INFO] Записано {length_out} позиций в {file_path_plus}")
            else:
                print("[WARNING] канал вне диапазона center_pred_plus.shape[1]")

            # - цепь
            if channel_ix < center_pred_minus.shape[1]:
                vals_minus = center_pred_minus[:, channel_ix]
                file_path_minus = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_st-_{bed_name}.bedGraph")

                with open(file_path_minus, "a") as fm:
                    for i, val in enumerate(vals_minus):
                        pos_start = cstart + i*stride
                        pos_end   = pos_start + stride
                        if pos_end > cend:
                            pos_end = cend
                        if pos_end <= pos_start:
                            break
                        fm.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

                #print(f"[INFO] Записано {length_out} позиций в {file_path_minus}")
            else:
                print("[WARNING] канал вне диапазона center_pred_minus.shape[1]")

    elapsed = time.time() - start_time
    print(f"Инференс завершён за {elapsed:.2f} секунд.")


In [21]:
# --- ЯЧЕЙКА 9: Запуск инференса ---

print("Start inference...")

run_inference(all_windows, models)

print("Done. 10 bedGraph files (5 тканей × 2 цепи) have been saved.")


Start inference...
Начинаем инференс по 1785 окнам.


Inference windows:  65%|█████████████████████████████████████████████████████████████████████████████████                                           | 1167/1785 [39:42<21:01,  2.04s/it]


KeyboardInterrupt: 